In [208]:
import os
import fnmatch
from textblob import TextBlob
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import pos_tag,pos_tag_sents
import re
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split  
from sklearn import metrics
from sklearn import svm
import pickle
from nltk.corpus import stopwords
import numpy as np
# import tensorflow as tf
# from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import string
import re
import matplotlib.pyplot as plt
import math
from matplotlib import rc
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
from bs4 import BeautifulSoup
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords


C:\Users\bcarniel\AppData\Local\Continuum\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bcarniel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [201]:
df = pd.read_csv(r"./finaldfs/masterdf_scrape.csv",index_col = 0)
df2 = pd.read_csv(r"./finaldfs/masterdf2_scrape - Copie.csv",index_col = 0)
df3 = pd.read_csv(r"./finaldfs/masterdf3_scrape.csv",index_col = 0)
df4 = pd.read_csv(r"./finaldfs/masterdf3_scrape_new.csv",index_col = 0)
df5 = pd.read_csv(r"./finaldfs/masterdf3_scrape.csv",index_col = 0)


df

,comment,rating,bonus_info,city
0,Très bon restaurant. La cuisine est délicieuse...,5.0,Détails FOURCHETTE DE PRIX 3 154 AMD - 5 256 A...,Paris
1,Restaurant qui n'est pas très grand (20-30 pla...,5.0,Détails FOURCHETTE DE PRIX 3 154 AMD - 5 256 A...,Paris
2,Très bonne cuisine avec des produits frais. No...,5.0,Détails FOURCHETTE DE PRIX 3 154 AMD - 5 256 A...,Paris
3,J'habite le quartier je suis entré ici un peu ...,5.0,Détails FOURCHETTE DE PRIX 3 154 AMD - 5 256 A...,Paris
4,Dans un cadre moderne et l’esprit fast food. M...,5.0,Détails FOURCHETTE DE PRIX 3 154 AMD - 5 256 A...,Paris
...,...,...,...,...
5,A faire c'était vraiment bon je recommande Tou...,5.0,Détails FOURCHETTE DE PRIX 5 256 AMD - 10 512 ...,Paris
6,Il y a la queue mais franchement ce n'est pas ...,2.0,Détails FOURCHETTE DE PRIX 5 256 AMD - 10 512 ...,Paris
7,On nous donne un ticket pour faire la queue ca...,4.0,Détails FOURCHETTE DE PRIX 5 256 AMD - 10 512 ...,Paris
8,Très bon classique quasiment pas d'attente et ...,5.0,Détails FOURCHETTE DE PRIX 5 256 AMD - 10 512 ...,Paris


In [202]:
df = df.append(df2)
df = df.append(df3)
df= df.append(df4)
df = df.append(df5)

In [203]:
len(df)

103200

In [204]:
df.loc[df["rating"]>=4,"sentiment"] = "Positive"
df.loc[df["rating"]==3,"sentiment"] = "Neutral"
df.loc[df["rating"]<=2,"sentiment"] = "Negative"


In [206]:
df.to_csv("df_all_comments.csv")

In [205]:
df.sentiment.value_counts()

Positive    94890
Neutral      5221
Negative     3089
Name: sentiment, dtype: int64

In [207]:
tempdf = df.loc[df["sentiment"] == "Positive"].sample(5000)
df_resampled = df[df["sentiment"]!="Positive"]
df_resampled = df_resampled.append(tempdf)
df_resampled.to_csv("resampled_comments.csv")

In [18]:
df_resampled 

,comment,rating,bonus_info,city,sentiment
1,Accueil sympa pas mal d’attente au final le ke...,3.0,Détails FOURCHETTE DE PRIX 2 895 AMD - 12 061 ...,Paris,Neutral
7,L’accueil est bonne mais les sandwichs mauvais...,1.0,Détails FOURCHETTE DE PRIX 2 895 AMD - 12 061 ...,Paris,Negative
1,En balade dans le quartier nous cherchons un k...,3.0,Détails FOURCHETTE DE PRIX 2 895 AMD - 12 061 ...,Paris,Neutral
6,kebab identique à ailleurs cela reste un kebab...,3.0,Détails FOURCHETTE DE PRIX 2 895 AMD - 12 061 ...,Paris,Neutral
5,Au moment où je rédige cet avis le restaurant ...,1.0,Détails FOURCHETTE DE PRIX 2 895 AMD - 12 061 ...,Paris,Negative
...,...,...,...,...,...
0,Bel endroit petit mais intime. Jolie décoratio...,5.0,Détails FOURCHETTE DE PRIX 1 577 AMD - 14 717 ...,Paris,Positive
0,Les plats mitonnés avec des produits frais vou...,5.0,Détails FOURCHETTE DE PRIX 4 731 AMD - 7 884 A...,Paris,Positive
3,l'un des meilleur Restaurant Iranienne a Paris...,5.0,Détails CUISINES Moyen-Orient Perse Grillades ...,Paris,Positive
1,Si vous voulez manger une choucroute ou des fr...,4.0,Détails FOURCHETTE DE PRIX 5 256 AMD - 13 140 ...,Paris,Positive


In [20]:
df_resampled.comment.to_list()[:10]

['Accueil sympa pas mal d’attente au final le kebab est correct sans être exceptionnel. Seulement deux places assises au comptoir il vaut mieux prendre à emporter.',
 'L’accueil est bonne mais les sandwichs mauvais de plus il se trompe de commande Je déconseille De plus il dis à ses clients de mettre 5 étoiles inadmissible voilà pourquoi ce classement honteuxPlus',
 'En balade dans le quartier nous cherchons un kebab et voici que Tripadvisor nous indique un restaurant dans le coin ET dans le TOP 20 des meilleurs endroits de Paris. Tout cela est bien sûr intriguant et voilà ma curiosité piquée. Déception le lieu sert...Plus',
 'kebab identique à ailleurs cela reste un kebab pas transcendant !! mais tout ces avis positif avec un seul avis par personne ou avez vous trouvé c est clients imaginaire en plus autant ???Plus',
 "Au moment où je rédige cet avis le restaurant est classé 11eme sur TripAdvisor. Je ne me suis pas arrêté a simplement croire que c'était des personnes sans aucun jugeme

In [64]:
df_resampled = pd.read_csv("resampled_comments.csv",index_col = 0)
df_resampled

,comment,rating,bonus_info,city,sentiment
1,Accueil sympa pas mal d’attente au final le ke...,3.0,Détails FOURCHETTE DE PRIX 2 895 AMD - 12 061 ...,Paris,Neutral
7,L’accueil est bonne mais les sandwichs mauvais...,1.0,Détails FOURCHETTE DE PRIX 2 895 AMD - 12 061 ...,Paris,Negative
1,En balade dans le quartier nous cherchons un k...,3.0,Détails FOURCHETTE DE PRIX 2 895 AMD - 12 061 ...,Paris,Neutral
6,kebab identique à ailleurs cela reste un kebab...,3.0,Détails FOURCHETTE DE PRIX 2 895 AMD - 12 061 ...,Paris,Neutral
5,Au moment où je rédige cet avis le restaurant ...,1.0,Détails FOURCHETTE DE PRIX 2 895 AMD - 12 061 ...,Paris,Negative
...,...,...,...,...,...
0,Super restaurant proche des Champs Élysées acc...,5.0,Détails FOURCHETTE DE PRIX 15 768 AMD - 31 537...,Paris,Positive
9,Nous sommes tombés sur cet hôtel sur Trip Advi...,5.0,Détails CUISINES Italienne Française Pizza Eur...,Paris,Positive
6,Diner pour mon anniversaire nous partons évide...,5.0,Emplacement et coordonnées 5 rue Coq Heron 750...,Paris,Positive
0,Tout est top ! Quand on arrive on est super bi...,5.0,Détails CUISINES Française Steakhouse Européen...,Paris,Positive


In [300]:
from nltk.stem.snowball import SnowballStemmer


df = pd.read_csv("resampled_comments.csv",index_col = 0)

def basic_preprocess(string):
    string = string.replace("Plus",'')
    lst_to_remove = [".","!","?","/","\'", "’"]         
    string = string.lower()
    for element in lst_to_remove:
        string = string.replace(element,' ')
    return string


def remove_useless_words(string):
    lst_useless_words = [" le "," la "," dans "," je "," nous "," en "," les "," cet "," ce "," cette ",
                         " de "," des "," un "," une "," à "," au ", "et", " il ", " ça ", " c ",
                        " sur ", " son ", " sa ", " me ", " par ", " ces ", " que "," je ","  ", " ainsi "]
    for element in lst_useless_words:
        string = string.replace(element,' ')
    return string

def tokenize(string):
    toknizer = RegexpTokenizer(r'''\w'|\w+|[^\w\s]''')
    string = toknizer.tokenize(string)
    return string

# def lemmatize(string):
#     lemmatizer = spacy.load('fr_core_news_md')
#     string=lemmatizer.lemmatize(string)
# https://stackoverflow.com/questions/13131139/lemmatize-french-text
#     return string

def stem(string):
    stemmer = SnowballStemmer(language='french')
    string = stemmer.stem(string)
    return string




df["comment"] = df["comment"].apply(basic_preprocess)
df["comment"] = df["comment"].apply(remove_useless_words)
df["comment"] = df["comment"].apply(stem)

# lemmatizer = FrenchLefffLemmatizer()
# df['comment'] = df['comment'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))


In [290]:
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk import RegexpTokenizer

df = pd.read_csv("resampled_comments.csv",index_col = 0)
# df = df[df["sentiment"]!="Neutral"]
## a essayer : tf-idf, tokenization, lemmatize


## lemmatize vs stem : no diff (maybe stem slightly better?), tf-idf better than countvectorizer, 


## Change the reviews type to string
df['comment'] = df['comment'].astype(str)

# Tokenize
# toknizer = RegexpTokenizer(r'''\w'|\w+|[^\w\s]''')
# df['comment'] = df['comment'].apply(lambda row: toknizer.tokenize(row['comment']),axis= 1)


## Lowercase all reviews
df['comment'] = df['comment'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['comment']

## remove punctuation
df['comment'] = df['comment'].str.replace('[^\w\s]','')
df['comment']

## Stop Words
stop = stopwords.words('french')
df['comment'] = df['comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['comment']

## Stemming
# st = SnowballStemmer(language='french')
# df['comment'] = df['comment'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
# df['comment']


## Lemmatizing
# from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
lemmatizer = FrenchLefffLemmatizer()
df['comment'] = df['comment'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))


In [312]:
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

X = df["comment"]
y = df["sentiment"]

Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,y,test_size = .2)


tfidf_vectorizer = TfidfVectorizer(max_df = 0.7 , min_df = 0.01 , max_features = 1000,ngram_range=(1, 3)) 
tfidf_vectorizer.fit(Xtrain)
Xtrain = cv.transform(Xtrain)
Xtest = cv.transform(Xtest)


# cv = CountVectorizer(binary=True, ngram_range=(1,3))
# cv.fit(Xtrain)
# Xtrain = cv.transform(Xtrain)
# Xtest = cv.transform(Xtest)



In [310]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split,cross_val_score
import warnings
warnings.filterwarnings("ignore")


for c in [0.05, 0.5, 2]:
    lr = LogisticRegression(C=c)
    lr.fit(Xtrain, Ytrain)
    scores = cross_val_score(lr, Xtrain, Ytrain, cv=5)
    print("c : ", c, "crossval accuracy : ", np.mean(scores))
#     print(scores_mnb2, "Avg accuracy MultinomialNB matrix: {}".format(np.mean(scores_mnb2)))
#     print ("Accuracy for C=%s: %s"
#            % (c, accuracy_score(Ytest, lr.predict(Xtest))))

c :  0.05 crossval accuracy :  0.8194966448132982
c :  0.5 crossval accuracy :  0.828136994820024
c :  2 crossval accuracy :  0.8258831649675727


In [313]:
from sklearn.naive_bayes import MultinomialNB

mnb2 = MultinomialNB()
mnb2.fit(Xtrain, Ytrain)
scores_mnb2 = cross_val_score(mnb2, Xtrain, Ytrain, cv=5)
print(scores_mnb2, "Avg accuracy MultinomialNB: {}".format(np.mean(scores_mnb2)))

[0.82065728 0.80610329 0.8028169  0.80601221 0.81117896] Avg accuracy MultinomialNB matrix: 0.8093537268703817


In [327]:
from sklearn import svm

SVM = svm.SVC(C=1.0, kernel='linear', gamma='auto',probability=True)
SVM.fit(Xtrain, Ytrain)
# scores_svm = cross_val_score(SVM, Xtrain, Ytrain, cv=5)
# print(scores_svm, "Avg accuracy SVM : {}".format(np.mean(scores_svm)))



SVC(gamma='auto', kernel='linear', probability=True)

In [325]:
from sklearn.ensemble import VotingClassifier

estimators=[("SVM", SVM), ("NB", mnb2), ("log_reg", lr)]

ensemble = VotingClassifier(estimators, voting="hard")

ensemble.fit(Xtrain, Ytrain)
predictions = ensemble.predict(Xtest)
print(accuracy_score(predictions,Ytest))


0.8414725770097671


In [328]:
from sklearn.ensemble import VotingClassifier

estimators=[("SVM", SVM), ("NB", mnb2), ("log_reg", lr)]

ensemble = VotingClassifier(estimators, voting="soft")

ensemble.fit(Xtrain, Ytrain)
predictions = ensemble.predict(Xtest)
print(accuracy_score(predictions,Ytest))


0.8467317806160781


In [329]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier

estimators=[("SVM", SVM), ("NB", mnb2), ("log_reg", lr)]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
clf.fit(Xtrain, Ytrain).score(Xtest, Ytest)



0.8456048084147257

In [330]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=1000, max_depth = 10,min_samples_leaf = 10)
clf.fit(Xtrain, Ytrain).score(Xtest, Ytest)

0.8437265214124718

In [331]:
from sklearn.ensemble import VotingClassifier

estimators=[("SVM", SVM), ("NB", mnb2), ("log_reg", lr), ("rf",rf)]

ensemble = VotingClassifier(estimators, voting="soft")

ensemble.fit(Xtrain, Ytrain)
predictions = ensemble.predict(Xtest)
print(accuracy_score(predictions,Ytest))


0.8471074380165289


In [332]:
estimators=[("SVM", SVM), ("NB", mnb2), ("log_reg", lr), ("rf",rf)]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
clf.fit(Xtrain, Ytrain).score(Xtest, Ytest)


0.8444778362133734

In [335]:
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier(base_estimator=mnb2,n_estimators=10)
clf.fit(Xtrain, Ytrain).score(Xtest, Ytest)



0.8264462809917356

In [336]:
clf = BaggingClassifier(base_estimator=lr,n_estimators=10)
clf.fit(Xtrain, Ytrain).score(Xtest, Ytest)


0.8444778362133734

In [337]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), lr.coef_[0]
    )
}
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:5]:
    print (best_positive)

for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:5]:
    print (best_negative)


('médiocre', 1.370492345777166)
('mauvais', 1.0489188633096849)
('décevant', 1.0187383448495495)
('travail sympathique', 1.0041274579843755)
('mauvaise', 0.9465170905364599)
('rapide', -0.9287199777693992)
('délicieux', -0.8701423212625325)
('parfait', -0.843440924510212)
('top', -0.7366676373945611)
('excellent', -0.7199798953180998)
